In [2]:
import pandas as pd
import numpy as np
import os
import re

In [3]:
# 找到TSGym的结果
arg_component_balance=False
arg_add_new_dataset=False
arg_add_transformer=True

result_path = '/data/nishome/user1/minqi/TSGym/meta/results'
datasets = ['ETTh1', 'ETTh2', 'ETTm1', 'ETTm2', 'ili', 'weather', 'ECL', 'Exchange', 'traffic']
# datasets = ['ETTh1', 'ETTh2', 'ETTm1', 'ETTm2', 'ili', 'weather', 'Exchange']
if arg_add_new_dataset:
    datasets.extend(['covid-19', 'fred-md'])

pred_len_1, pred_len_2 = 96, 24
file_list = [f'{dataset}-component_balance_{arg_component_balance}-add_new_dataset_{arg_add_new_dataset}-add_transformer_{arg_add_transformer}_{pred_len_1}_{pred_len_2}.npz' for dataset in datasets]



In [4]:
dfs_dict = {}
for pred_len_1, pred_len_2 in zip([96, 192, 336, 720], [24, 36, 48, 60]):
    file_list = [f'{dataset}-component_balance_{arg_component_balance}-add_new_dataset_{arg_add_new_dataset}-add_transformer_{arg_add_transformer}_{pred_len_1}_{pred_len_2}.npz' for dataset in datasets]
    for i, file in enumerate(file_list):
        dataset = file[:re.search('-component', file).start()]
        perf_epoch = np.load(os.path.join(result_path, file), allow_pickle=True)
        top1_perf_epoch = perf_epoch['top1_perf_epoch']
        if pred_len_1 == 96:
            dfs_dict[dataset] = []
        dfs_dict[dataset].append(top1_perf_epoch[perf_epoch['best_epoch'].item()])

FileNotFoundError: [Errno 2] No such file or directory: '/data/nishome/user1/minqi/TSGym/meta/results/ETTh1-component_balance_False-add_new_dataset_False-add_transformer_True_96_24.npz'

In [ ]:
for dataset in dfs_dict.keys():
    dfs_dict[dataset] = sum(dfs_dict[dataset]) / len(dfs_dict[dataset])
dfs_dict

{'ETTh1': 0.41929612308740616,
 'ETTh2': 0.36302248388528824,
 'ETTm1': 0.364893801510334,
 'ETTm2': 0.2680737264454365,
 'ili': 2.2638270258903503,
 'weather': 0.2282826118171215,
 'ECL': 0.17199154570698738,
 'Exchange': 0.42001335322856903,
 'traffic': 0.4149283319711685}

In [8]:
# 找到SOTA的结果
sota_baseline_list = ['DUET','TimeMixer','TSMixer','MICN','TimesNet','PatchTST','Crossformer','SegRNN','Pyraformer','Autoformer']

In [5]:
# sota performance
def search_sota_performance(dataset, pred_lens=96,
                            path_old='/data/nishome/user1/minqi/TSGym/results', 
                            path='/data/nishome/user1/minqi/TSGym/results_long_term_forecasting/results',
                            metrics='mse'):
    result_dict = {}
    model_list_old = os.listdir(path_old)
    model_list_old = [_ for _ in model_list_old if f'pl{pred_lens}' in _ and dataset in _]

    try:
        model_list_new = os.listdir(os.path.join(path, dataset))
        model_list_new = [_ for _ in model_list_new if f'pl{pred_lens}' in _]
    except:
        model_list_new = []
    model_list = model_list_old + model_list_new

    result_dict[pred_lens] = {}
    for model in model_list:
        try:
            result = np.load(os.path.join(path_old, model, 'metrics.npy'), allow_pickle=True)
        except:
            result = np.load(os.path.join(path, dataset, model, 'metrics.npy'), allow_pickle=True)
        if metrics == 'mse':
            result_dict[pred_lens][model] = result[1]
        elif metrics == 'mae':
            result_dict[pred_lens][model] = result[0]
        else:
            raise NotImplementedError

    df = pd.DataFrame.from_dict(result_dict[pred_lens], orient='index')
    if df.shape[0] < 1:
        print(dataset)
        print(result_dict)
    df.columns = ['mse']
    df.index = [_.split('_')[1] if 'LTF' in _ or 'STF' in _ else _.split('_')[6] for _ in df.index]
    # 有重复值
    df = df[~df.index.duplicated(keep='first')]
    df = df[df.index.isin(sota_baseline_list)]
    df = df.sort_index()
    return df

In [6]:
def run_get_sota(pred_lens1,pred_lens2,metrics):
    sota_performance = []
    for dataset in ['ETTh1','ETTh2','ETTm1','ETTm2','ili','weather','ECL','Exchange','traffic']:
        if dataset == 'ili':
            pred_lens = pred_lens1 # 24,36,48,60
        else:
            pred_lens = pred_lens2 # 96,192,336,720
        result = search_sota_performance(dataset,pred_lens,metrics=metrics)
        result.columns=[dataset]
        sota_performance.append(result)
    results = pd.concat(sota_performance, axis=1)
    results = results
    results = results.reindex(sota_baseline_list)
    return results

In [11]:
## 计算MSE
metrics='mse'

result_sota_96 = pd.read_csv('/data/nishome/user1/minqi/TSGym/results/result_SOTA_96_96.csv')
result_sota_96 = result_sota_96[['Model','ETTh1','ETTh2','ETTm1','ETTm2','ili','weather','ECL','Exchange','traffic']]
result_sota_96 = result_sota_96[result_sota_96.Model.isin(sota_baseline_list)]
result_sota_96 = result_sota_96.set_index('Model')
result_sota_96 = result_sota_96.reindex(sota_baseline_list)
result_sota_96

result_sota_336 = run_get_sota(48,336,metrics=metrics)
result_sota_192 = run_get_sota(36,192,metrics=metrics)
result_sota_720 = run_get_sota(60,720,metrics=metrics)
result_sota_mse = (result_sota_96+result_sota_192+result_sota_336+result_sota_720)/4
result_sota_mse.T

Model,DUET,TimeMixer,TSMixer,MICN,TimesNet,PatchTST,Crossformer,SegRNN,Pyraformer,Autoformer
ETTh1,0.440365,0.447838,0.614810,0.592510,0.473635,0.457469,0.546491,0.421863,0.814491,0.495945
ETTh2,0.379705,0.389572,2.163829,0.585259,0.427304,0.385807,1.661696,0.374222,3.802302,0.462128
ETTm1,0.407312,0.391973,0.529003,0.403069,0.432329,0.394034,0.518420,0.388861,0.708374,0.531958
ETTm2,0.295738,0.278108,1.030511,0.342194,0.295691,0.290034,1.805239,0.272607,1.622015,0.421966
ili,2.639685,4.501613,5.616794,2.970958,2.227669,2.160230,4.310894,4.304993,4.691041,3.156397
weather,0.252006,0.244053,0.242810,0.265444,0.260664,0.258759,0.260876,0.251509,0.283978,0.341100
ECL,0.178751,0.184164,0.232337,0.188661,0.218546,0.208632,0.196155,0.572611,0.294868,0.237983
Exchange,0.322428,0.358731,0.487187,0.365537,0.405259,0.385236,0.903970,0.407919,1.252989,0.520271
traffic,NaN,1.372002,0.594782,0.543786,0.955951,0.496685,NaN,0.770211,1.251442,0.637542


In [12]:
# 计算MAE

metrics='mae'

result_sota_96 = run_get_sota(24,96,metrics=metrics)

result_sota_336 = run_get_sota(48,336,metrics=metrics)
result_sota_192 = run_get_sota(36,192,metrics=metrics)
result_sota_720 = run_get_sota(60,720,metrics=metrics)
result_sota_mae = (result_sota_96+result_sota_192+result_sota_336+result_sota_720)/4
result_sota_mae.T

,DUET,TimeMixer,TSMixer,MICN,TimesNet,PatchTST,Crossformer,SegRNN,Pyraformer,Autoformer
ETTh1,0.446290,0.438220,0.579415,0.538917,0.463733,0.451110,0.521382,0.428956,0.692101,0.487661
ETTh2,0.405667,0.407565,1.220570,0.529773,0.432929,0.409419,0.953324,0.404877,1.559484,0.470129
ETTm1,0.409264,0.400806,0.514177,0.429781,0.429707,0.403743,0.507940,0.404588,0.601174,0.495872
ETTm2,0.337534,0.325453,0.749973,0.390977,0.334206,0.334258,0.837852,0.321720,0.893457,0.439751
ili,1.017726,1.557109,1.679857,1.185897,0.925865,0.901482,1.396064,1.396647,1.442438,1.207956
weather,0.276767,0.273968,0.301318,0.315850,0.286537,0.280096,0.324501,0.298271,0.348554,0.381605
ECL,0.262228,0.272985,0.337408,0.299825,0.314217,0.298512,0.290593,0.302398,0.386689,0.342014
Exchange,0.384005,0.401929,0.545910,0.437184,0.436703,0.414970,0.697089,0.423239,0.887226,0.507442
traffic,NaN,0.313301,0.403449,0.320565,0.503080,0.320605,NaN,0.410596,0.664018,0.397456


In [14]:
# 重命名列名以区分 MSE 和 MAE
mse_renamed = result_sota_mse.T.rename(columns=lambda x: f"{x}_MSE")
mae_renamed = result_sota_mae.T.rename(columns=lambda x: f"{x}_MAE")
combined_df = pd.concat([mse_renamed, mae_renamed], axis=1)
# 按模型排序列（可选）
ordered_columns = []
for baseline in sota_baseline_list:
    ordered_columns.extend([f"{baseline}_MSE", f"{baseline}_MAE"])
combined_df = combined_df[ordered_columns]
combined_df

,DUET_MSE,DUET_MAE,TimeMixer_MSE,TimeMixer_MAE,TSMixer_MSE,TSMixer_MAE,MICN_MSE,MICN_MAE,TimesNet_MSE,TimesNet_MAE,PatchTST_MSE,PatchTST_MAE,Crossformer_MSE,Crossformer_MAE,SegRNN_MSE,SegRNN_MAE,Pyraformer_MSE,Pyraformer_MAE,Autoformer_MSE,Autoformer_MAE
ETTh1,0.440365,0.446290,0.447838,0.438220,0.614810,0.579415,0.592510,0.538917,0.473635,0.463733,0.457469,0.451110,0.546491,0.521382,0.421863,0.428956,0.814491,0.692101,0.495945,0.487661
ETTh2,0.379705,0.405667,0.389572,0.407565,2.163829,1.220570,0.585259,0.529773,0.427304,0.432929,0.385807,0.409419,1.661696,0.953324,0.374222,0.404877,3.802302,1.559484,0.462128,0.470129
ETTm1,0.407312,0.409264,0.391973,0.400806,0.529003,0.514177,0.403069,0.429781,0.432329,0.429707,0.394034,0.403743,0.518420,0.507940,0.388861,0.404588,0.708374,0.601174,0.531958,0.495872
ETTm2,0.295738,0.337534,0.278108,0.325453,1.030511,0.749973,0.342194,0.390977,0.295691,0.334206,0.290034,0.334258,1.805239,0.837852,0.272607,0.321720,1.622015,0.893457,0.421966,0.439751
ili,2.639685,1.017726,4.501613,1.557109,5.616794,1.679857,2.970958,1.185897,2.227669,0.925865,2.160230,0.901482,4.310894,1.396064,4.304993,1.396647,4.691041,1.442438,3.156397,1.207956
weather,0.252006,0.276767,0.244053,0.273968,0.242810,0.301318,0.265444,0.315850,0.260664,0.286537,0.258759,0.280096,0.260876,0.324501,0.251509,0.298271,0.283978,0.348554,0.341100,0.381605
ECL,0.178751,0.262228,0.184164,0.272985,0.232337,0.337408,0.188661,0.299825,0.218546,0.314217,0.208632,0.298512,0.196155,0.290593,0.572611,0.302398,0.294868,0.386689,0.237983,0.342014
Exchange,0.322428,0.384005,0.358731,0.401929,0.487187,0.545910,0.365537,0.437184,0.405259,0.436703,0.385236,0.414970,0.903970,0.697089,0.407919,0.423239,1.252989,0.887226,0.520271,0.507442
traffic,NaN,NaN,1.372002,0.313301,0.594782,0.403449,0.543786,0.320565,0.955951,0.503080,0.496685,0.320605,NaN,NaN,0.770211,0.410596,1.251442,0.664018,0.637542,0.397456
